# 이미지 구축 코드

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import BytesIO
from PIL import Image as Img
from tqdm import tqdm
import time

In [2]:

# 카테고리 분류 기록
# 실제카테고리(효율적인 검색어) : 타켓 카테고리 


# 예외처리를 위해 세컨드웨어의 분류 키워드 활용 -> 검색 결과 중 세컨드 웨어 상 카테고리가 리스트 안에 있는 것만 가져오기 위함.

categories = {
'상의' : {
    '니트' : ['니트/스웨터'],                                                   
    '후드' : ['후드티셔츠'],
    '맨투맨' : ['맨투맨/스웻셔츠'] ,
    '와이셔츠'  : ['셔츠', '셔츠/블라우스'],
    '블라우스' : ['셔츠', '셔츠/블라우스'],
    '롱슬리브' : ['티셔츠', '맨투맨/스웻셔츠', '스포츠 상의'],
    '반팔' : ['티셔츠' , '스포츠 상의'],                                                    
    '나시티' : [],                                                                             
    '카라티셔츠' : ['티셔츠', '골프웨어 상의'],
    '니트베스트' : ['조끼', '니트/스웨터', '골프 용품', '골프웨어 상의'],
    },
'하의' : {
    '청바지' : ['바지'],
    '슬랙스' : ['바지', '슬랙스'],
    '트레이닝바지' : ['스포츠 하의' , '스웻팬츠', '바지'],
    '조거팬츠' : ['스포츠 하의' , '스웻팬츠', '바지'],
    '반바지' : ['반바지', '바지', '골프웨어 하의', '스포츠 하의', '스웻팬츠'],
    '면바지' : ['바지', '골프웨어 하의'], 
    '레깅스' : ['바지', '스포츠 하의', '스포츠 웨어', '팬츠/레깅스']
    },
'아우터' : {
    '후드집업' : ['후드티셔츠', '점퍼/자켓'],
    '바람막이' : ['점퍼/자켓', '스포츠 상의'],
    '코트' : ['코트'],                                                                                
    '롱패딩' : ['패딩', '점퍼/자켓'],
    '숏패딩' : ['패딩', '점퍼/자켓'],
    '패딩조끼' : ['조끼', '패딩'],
    '블루종' : ['점퍼/자켓'],
    '가죽자켓' :['점퍼/자켓'],
    '무스탕' : ['점퍼/자켓', '코트'],
    '트러커자켓' : ['점퍼/자켓' ],                                                 
    '블레이저' : ['점퍼/자켓', '정장'],
    '가디건' :[ '카디건', '니트/스웨터'],
    '뽀글이후리스' : ['점퍼/자켓','후드티셔츠','코트', '카디건'],
    '사파리자켓' :[ '점퍼/자켓']
    },
'원피스' : {
    '미니원피스' : ['원피스'  ],
    '미디원피스' :[ '원피스'],
    '롱원피스' : ['원피스']
},
'스커트' : {
    '미니스커트' :[ '스커트'],
    '미디스커트' :[ '스커트'],
    '롱스커트' : ['스커트']
}
}

matching = {
    '니트' : '니트', 
    '후드' : '후드', 
    '맨투맨' : '맨투맨',
    '와이셔츠' : '셔츠블라우스', 
    '블라우스' : '셔츠블라우스',
    '롱슬리브' : '긴소매티셔츠',
    '반팔' : '반소매티셔츠',
    '나시티' : '민소매티셔츠',
    '카라티셔츠' : '카라티셔츠',
    '니트베스트' : '베스트',
    '청바지' : '데님팬츠', 
    '슬랙스' : '슬랙스',
    '트레이닝바지' : '트레이닝조거팬츠',
    '조거팬츠' : '트레이닝조거팬츠',
    '반바지' : '숏팬츠',
    '면바지' : '코튼팬츠',
    '레깅스' : '레깅스',
    '후드집업' : '후드집업', 
    '바람막이' : '바람막이', 
    '코트' : '코트', 
    '롱패딩' : '롱패딩', 
    '숏패딩' : '숏패딩', 
    '패딩조끼' : '패딩베스트', 
    '블루종' : '블루종',                 
    '가죽자켓' : '레더자켓', 
    '무스탕' : '무스탕', 
    '트러커자켓' : '트러커자켓', 
    '블레이저' : '블레이저', 
    '가디건' : '가디건',
    '뽀글이후리스' : '뽀글이후리스',
    '사파리자켓' : '사파리자켓',
    '미니원피스' : '미니원피스', 
    '미디원피스' : '미디원피스', 
    '롱원피스' : '롱원피스',
    '미니스커트' : '미니스커트', 
    '미디스커트' : '미디스커트', 
    '롱스커트' : '롱스커트'
}

In [19]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}
count = 100

image = []
for main, f_dict in tqdm(categories.items()) :
    for sub, keyword in f_dict.items() :
        sub_composition = []
        for page in range(1, 11) :
            print(f'{sub}_{page}페이지 크롤링 중...')
            try : 
                url = f'https://www.hellomarket.com/api/search/items?q={sub}&page={page}&limit={count}'
                res = requests.get(url, headers=headers)
                all_info_dict = res.json()
                
                # 정상적으로 불러와졌다면
                if len(all_info_dict['list']) == count :
                    # 제품 하나 하나에 접근해서 특정 카테고리인 것이면 이미지 url을 가지고 오기
                    image_urls = [product['imageUrl'] for product in all_info_dict['list'] if product['categories'][1]['name'] in keyword]
                    sub_composition.extend(image_urls)
            except :
                print(f'{sub}_{page}페이지 오류 발생')
        
        # 한 서브마다 이미지 저장하기
        for i, image_url in enumerate(sub_composition) :
            try :
                response = requests.get(image_url)  
                img = Img.open(BytesIO(response.content))
                img.save(f'./real_dataset/{main}_{matching[sub]}_image_{i}.jpg')
            except :
                pass
        
        print(f'{sub} 크롤링 완료')   
    print(f'{main} 크롤링 완료')
        

  0%|          | 0/5 [00:00<?, ?it/s]

니트_1페이지 크롤링 중...
니트_2페이지 크롤링 중...
니트_3페이지 크롤링 중...
니트_4페이지 크롤링 중...
니트_5페이지 크롤링 중...
니트_6페이지 크롤링 중...
니트_7페이지 크롤링 중...
니트_8페이지 크롤링 중...
니트_9페이지 크롤링 중...
니트_10페이지 크롤링 중...
니트 크롤링 완료
후드_1페이지 크롤링 중...
후드_2페이지 크롤링 중...
후드_3페이지 크롤링 중...
후드_4페이지 크롤링 중...
후드_5페이지 크롤링 중...
후드_6페이지 크롤링 중...
후드_7페이지 크롤링 중...
후드_8페이지 크롤링 중...
후드_9페이지 크롤링 중...
후드_10페이지 크롤링 중...
후드 크롤링 완료
맨투맨_1페이지 크롤링 중...
맨투맨_2페이지 크롤링 중...
맨투맨_3페이지 크롤링 중...
맨투맨_4페이지 크롤링 중...
맨투맨_5페이지 크롤링 중...
맨투맨_6페이지 크롤링 중...
맨투맨_7페이지 크롤링 중...
맨투맨_8페이지 크롤링 중...
맨투맨_9페이지 크롤링 중...
맨투맨_10페이지 크롤링 중...
맨투맨 크롤링 완료
와이셔츠_1페이지 크롤링 중...
와이셔츠_2페이지 크롤링 중...
와이셔츠_3페이지 크롤링 중...
와이셔츠_4페이지 크롤링 중...
와이셔츠_5페이지 크롤링 중...
와이셔츠_6페이지 크롤링 중...
와이셔츠_7페이지 크롤링 중...
와이셔츠_8페이지 크롤링 중...
와이셔츠_9페이지 크롤링 중...
와이셔츠_10페이지 크롤링 중...
와이셔츠 크롤링 완료
블라우스_1페이지 크롤링 중...
블라우스_2페이지 크롤링 중...
블라우스_3페이지 크롤링 중...
블라우스_4페이지 크롤링 중...
블라우스_5페이지 크롤링 중...
블라우스_6페이지 크롤링 중...
블라우스_7페이지 크롤링 중...
블라우스_8페이지 크롤링 중...
블라우스_9페이지 크롤링 중...
블라우스_10페이지 크롤링 중...
블라우스 크롤링 완료
롱슬리브_1페이지 크롤링 중...
롱슬리브_2페이지 크롤링 중...
롱슬

 20%|██        | 1/5 [37:36<2:30:25, 2256.33s/it]

니트베스트 크롤링 완료
상의 크롤링 완료
청바지_1페이지 크롤링 중...
청바지_2페이지 크롤링 중...
청바지_3페이지 크롤링 중...
청바지_4페이지 크롤링 중...
청바지_5페이지 크롤링 중...
청바지_6페이지 크롤링 중...
청바지_7페이지 크롤링 중...
청바지_8페이지 크롤링 중...
청바지_9페이지 크롤링 중...
청바지_10페이지 크롤링 중...
청바지 크롤링 완료
슬랙스_1페이지 크롤링 중...
슬랙스_2페이지 크롤링 중...
슬랙스_3페이지 크롤링 중...
슬랙스_4페이지 크롤링 중...
슬랙스_5페이지 크롤링 중...
슬랙스_6페이지 크롤링 중...
슬랙스_7페이지 크롤링 중...
슬랙스_8페이지 크롤링 중...
슬랙스_9페이지 크롤링 중...
슬랙스_10페이지 크롤링 중...
슬랙스 크롤링 완료
트레이닝바지_1페이지 크롤링 중...
트레이닝바지_2페이지 크롤링 중...
트레이닝바지_3페이지 크롤링 중...
트레이닝바지_4페이지 크롤링 중...
트레이닝바지_5페이지 크롤링 중...
트레이닝바지_6페이지 크롤링 중...
트레이닝바지_7페이지 크롤링 중...
트레이닝바지_8페이지 크롤링 중...
트레이닝바지_9페이지 크롤링 중...
트레이닝바지_10페이지 크롤링 중...
트레이닝바지 크롤링 완료
조거팬츠_1페이지 크롤링 중...
조거팬츠_2페이지 크롤링 중...
조거팬츠_3페이지 크롤링 중...
조거팬츠_4페이지 크롤링 중...
조거팬츠_5페이지 크롤링 중...
조거팬츠_6페이지 크롤링 중...
조거팬츠_7페이지 크롤링 중...
조거팬츠_8페이지 크롤링 중...
조거팬츠_9페이지 크롤링 중...
조거팬츠_10페이지 크롤링 중...
조거팬츠 크롤링 완료
반바지_1페이지 크롤링 중...
반바지_2페이지 크롤링 중...
반바지_3페이지 크롤링 중...
반바지_4페이지 크롤링 중...
반바지_5페이지 크롤링 중...
반바지_6페이지 크롤링 중...
반바지_7페이지 크롤링 중...
반바지_8페이지 크롤링 중...
반바지_9페이지 크롤링 중...
반바지

 40%|████      | 2/5 [1:11:16<1:45:51, 2117.17s/it]

...

 80%|████████  | 4/5 [2:27:34<34:36, 2076.56s/it]  

롱원피스 크롤링 완료
원피스 크롤링 완료
미니스커트_1페이지 크롤링 중...
미니스커트_2페이지 크롤링 중...
미니스커트_3페이지 크롤링 중...
미니스커트_4페이지 크롤링 중...
미니스커트_5페이지 크롤링 중...
미니스커트_6페이지 크롤링 중...
미니스커트_7페이지 크롤링 중...
미니스커트_8페이지 크롤링 중...
미니스커트_9페이지 크롤링 중...
미니스커트_10페이지 크롤링 중...
미니스커트 크롤링 완료
미디스커트_1페이지 크롤링 중...
미디스커트_2페이지 크롤링 중...
미디스커트_3페이지 크롤링 중...
미디스커트_4페이지 크롤링 중...
미디스커트_5페이지 크롤링 중...
미디스커트_6페이지 크롤링 중...
미디스커트_7페이지 크롤링 중...
미디스커트_8페이지 크롤링 중...
미디스커트_9페이지 크롤링 중...
미디스커트_10페이지 크롤링 중...
미디스커트 크롤링 완료
롱스커트_1페이지 크롤링 중...
롱스커트_2페이지 크롤링 중...
롱스커트_3페이지 크롤링 중...
롱스커트_4페이지 크롤링 중...
롱스커트_5페이지 크롤링 중...
롱스커트_6페이지 크롤링 중...
롱스커트_7페이지 크롤링 중...
롱스커트_8페이지 크롤링 중...
롱스커트_9페이지 크롤링 중...
롱스커트_10페이지 크롤링 중...


100%|██████████| 5/5 [2:43:42<00:00, 1964.54s/it]

롱스커트 크롤링 완료
스커트 크롤링 완료


: 

In [16]:
response = requests.get(sub_composition[2])
img = Img.open(BytesIO(response.content))
img.save('./temp.jpg')


# df 만드는 코드

In [ ]:

categories = {
    '상의' : ['니트', '후드', '맨투맨','셔츠블라우스','긴소매티셔츠','반소매티셔츠','민소매티셔츠','카라티셔츠','베스트'],
    '바지' : ['데님팬츠', '슬랙스','트레이닝조거팬츠','숏팬츠','코튼팬츠','레깅스'],
    '아우터' : ['후드집업', '바람막이', '코트', '롱패딩', '숏패딩', '패딩베스트', '블루종', 
                '레더자켓', '무스탕', '트러커자켓', '블레이저', '가디건','뽀글이후리스','사파리자켓'],
    '원피스' : ['미니원피스', '미디원피스', '롱원피스', '점프수트'],
    '스커트' : ['미니스커트', '미디스커트', '롱스커트']}



def secondwear_crawler(categories) :

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}

    csv_base_path = './db_data_to_insert_csv/'

    dataframes = []

    for main,subs  in categories.items() :
        for sub in subs :
            print(f'{sub} 크롤링 중 ...')
            title,price,connect_url,product_type,status,brand,top,bottom,color,size,product_url = [[] for _ in range(11)]
            for page in range(1, 9) :

                url = f'https://www.hellomarket.com/api/search/items?q={sub}&page={page}&limit=30'

                res = requests.get(url, headers=headers)
                all_info_dict = res.json()

                product_info_list = all_info_dict['list']

                for product_info in product_info_list :
                    # 제목
                    p_title = product_info['title']

                    # 상품가격
                    p_price = product_info['price']
                    
                    # 판매처 url
                    p_connect_url = product_info['linkUrl']
                    
                    # 상품타입
                    p_type = 0
                    
                    # 상품상태
                    p_status = product_info['usedType']['name']
                    
                    # 브랜드
                    try:
                        p_brand = product_info['brand']['korean']
                    except:
                        p_brand = 'NO BRAND'
                    # 상위 카테고리
                    p_top = f'{main}'
                    
                    # 하위 카테고리
                    p_bottom = f'{sub}'
                    
                    # 색깔
                    p_color = 'null'

                    # 사이즈
                    try :
                        p_size = product_info['clothesSize']['name']
                    except :
                        p_size = 'FREE'
                    
                    # 상품이미지 url
                    p_product_url = product_info['imageUrl']

                    # 반영
                    title.append(p_title)
                    price.append(p_price)
                    connect_url.append(p_connect_url)
                    product_type.append(p_type)
                    status.append(p_status)
                    brand.append(p_brand)
                    top.append(p_top)
                    bottom.append(p_bottom)
                    color.append(p_color)
                    size.append(p_size)
                    product_url.append(p_product_url)
            df = pd.DataFrame({
                'title' : title,
                'price' : price,
                'connect_url' : connect_url,
                'type' : product_type,
                'status' : status,
                'brand' : brand,
                'top_category' : top,
                'bottom_category' : bottom,
                'color' : color,
                'size' : size,
                'product_url' : product_url,
                })  

            dataframes.append(df)  


    final_df = pd.concat(dataframes, axis = 0, ignore_index=True)
    final_df['connect_url'] = [f'https://www.hellomarket.com/item/{connect_url.split("/")[-1]}?viewLocation=search_result' for connect_url in final_df['connect_url']]

    print(final_df)

    final_df.to_csv(csv_base_path + 'secondwear_data2.csv', index=False)   